# Tarea 1
#### Este equipo esta conformado por los siguientes integrantes:
- Mariscal Villalobos Cesar -  218206852
- Rivera Osuna Kevin Eduardo - 222216237

# Problema 1 - Modificando la intensidad de una imagen
Cuando hablamos sobre intensidad, nos referimos a los niveles de gris que hay en un pixel dado de nuestra imagen.
La operación de modificación de intensidad, nos modifica los valores originales de intensidad de los pixeles de la imagen, pudiendo escalar estos pixeles Sea $I$ nuestro arreglo de pixeles representantes de la imagen y sea $\alpha$ un escalar cualquiera, aumentamos la intensidad de nuestra imagen por un factor alpha como:  $I * \alpha $, o simplemente escalamos usando un cantidad fija de unidades, usando una suma $I + \beta$.

Debemos tener cuidado con los escalamientos, esto debido a que al momento de hacer los calculos, podemos desbordar nuestra escala de blancos y negros $[0,255]$. Por lo que una imagen con valores de intensidades cercanos a 255 puede que no sea un buen candidato para amplificar la intensidad.

In [1]:
import cv2 as cv
import numpy as np
from PIL import Image, ImageOps

### Ejemplo realizandeo un escalamiento por un factor constante a los pixeles de la imagen

In [2]:
# Comenzamos capturando la imagen que queremos procesar. Por lo general lo hacemos en escalas de grises
# ya que nos permite que el algoritmo capture lo escencial de la imagen y funcione mejor.
# Procesar una imagen a color puede agregar variaciones o complejidad innecesaria en ciertos casos.
image = cv.imread('Alex.jpg',cv.IMREAD_GRAYSCALE)

alpha = 1.5

# Aumentamos la intensidad de la imagen, debemos tener cuidado porque nuestra matriz que guardaba las intensidades
# los cuales son valores de 0 a 255, son numeros enteros, por lo que ahora tendremos una matriz de flotantes
image = image * alpha

# np.savetxt("matriz_float.txt", image, fmt="%d")
# Hacemos nuestra matriz de flotantes a enteros
image = image.astype(np.uint8)
# np.savetxt("matriz_post.txt", image, fmt="%d")

# Mostramos la imagen
cv.imshow('Image',image)
cv.waitKey(0)
cv.destroyAllWindows()



### Ejemplo realizando un escalamiento a la imagen sumando un entero a sus pixeles

In [3]:
image = cv.imread('Alex.jpg',cv.IMREAD_GRAYSCALE)

beta = 70

# Notemos que no ocupamos transformar nuestra matriz porque hicimos una suma de enteros.
image = image + beta

# Mostramos la imagen
cv.imshow('Image',image)
cv.waitKey(0)
cv.destroyAllWindows()

# Ejemplo realizando sobre escalamiento

#### Explicación del ejemplo:
Cuando nosotros aumentamos la intensidad, esperamos que la imagen se vuelva cada vez más blanca, debido a que los pixeles estan siendo escalados positivaente, sin embargo, si nos fijamos justo en el ojo de alex, hay secciones negras y blancas, en especifico hay una sección muy blanca en la iris de alex, la cual posteriormente al aumentar la intensidad por un factor de 2.1, esa sección que antes era muy blanca, en vez de quedarse en blanco se volvió negro, esto debido a un problema de overflow en nuestro intervalo de colores.


In [4]:
# leemos la imagen, simplemente el ojo de ALEX
image = cv.imread('Alex_eye.jpg',cv.IMREAD_GRAYSCALE)

alpha = 2.1

image_s = image * alpha

image_s = image_s.astype(np.uint8)

# Mostramos la imagen

cv.imshow('Image_scaled',image_s)
cv.waitKey(0)
cv.destroyAllWindows()


# Problema 2 - Invertir la intensidad de imagenes

Consiste en invertir la intensidad de una imagen, dicho esto, podemos resumir el efecto como:
- Si la imagen original es muy brillosa/blanca, esta se vuelve oscura/negra
- Si la imagen original es muy oscura, esta se vuelve brillosa.
- Si invertimos en escala de grises: $255 \rightarrow 0$
Sin embargo, para imagenes de color, al realizar una operación de invertir, lo que nos da como resultado son colores complementarios, por ejemplo:
- Si invertimos el color rojo: $(255,0,0) \rightarrow (0,255,255)$
- Si invertimos el color: $(0,0,0) \rightarrow (255,255,255)$
- Si invertimos el color: $(5,255,100) \rightarrow (250,0,155)$
Dada la intensidad $I$ del pixel $P$ que denotare como $I_p$, y dado la intensidad maxima $I_\text{max}=255$, la inversión de pixel se realiza como:
$$I_\text{invertida} = I_\text{max}-I_p$$
Esta formula funciona tanto para escala de grises como imagenes a color, sin embargo, si queremos aplicar la operación de inversión en una imagen rgb, debemos aplicarla en cada canal R G B.

Para este operador no encontramos un caso en el que no se deba de aplicar o tener cuidado, ya que simplemente se esta aplicando una inversión de colores y no representa un cambio muy conflictivo (ya que puede ser facilmente reversible)

### Caso 1 - Invertir en escala de grises

In [5]:
# mapeamos los pixeles de la imagen en una matriz
image = cv.imread("Castillo.jpg",cv.IMREAD_GRAYSCALE)

# bitwise_not lo que hace es que invierte los valores de los pixeles de la imagen a nivel de bits
# Ejemplo: 00101011 ---> 11010100
img_inv = cv.bitwise_not(image)

cv.imshow("Original_image",image)
cv.imshow("Inverted_image",img_inv)
cv.waitKey(0)
cv.destroyAllWindows()

### Caso 2 - Invertir imagen RGB

In [6]:
image = cv.imread("Castillo.jpg")

img_inv = cv.bitwise_not(image)

cv.imshow("Original_image",image)
cv.imshow("Inverted_image",img_inv)
cv.waitKey(0)
cv.destroyAllWindows()

# Problema 3 - Operador Umbral
Esta operación nos permite dar ciertos valores a un pixel, dependiendo si se cumple una condición.
Parece ser muy util para dividir o puntualizar ciertas zonas de la imagen, por ejemplo zonas con poca luz y zonas con mucha luz.
La operación mas que una forma aritmética, tiene una forma lógica, por lo que su definición se puede ver como el uso de una función if o una función partida la cuál asigna un valor dependiendo de cierto "Umbral".
Definimos dos valores constantes $a_0,a_1$, los cuales nos representan los posibles valores que puede asignar la función, tenemos también la constante $q$, la cual nos representara el "Umbral" o valor limite con el cual condicionaremos los valores, entonces:
$$ 
f_\text{threshold}(a) =
\begin{cases}
a_0 & \text{si } a < q  \\
a_1 & \text{si } a \ge q
\end{cases}
$$
Es interesante esto ya que las imagenes se pueden binarizar de una forma muy sencilla.
Hay que tener cuidado en seleccionar un buen umbral, debido a que seleccionar un umbral de intensidad muy alto, puede repercutir en perder toda la información.

##### Diferentes casos de uso:
Imagen completamente blanca con valor de pixeles $p=255$:
- Umbral $0 < q < 255$: La imagen no sufrira un cambio, debido a que el umbral siempre estara por debajo del valor máximo de intensidad
- Umbral $q = 255$: La imagen se convertira completamente negra, debido a que el umbral sería igual a la intensidad máxima.




### Aplicando un umbral de 30
Notamos como esta binarización nos detecta al personaje completamente, separando adecuadamente los diferentes niveles de intensidad.

In [7]:
image = cv.imread('Hornet.jpg',cv.IMREAD_GRAYSCALE)

_,image_umbral = cv. threshold(image, 30,
                             255,
                             cv.THRESH_BINARY)
cv.imshow('Original_image',image)
cv.imshow('Image_umbral',image_umbral)
cv.waitKey(0)
cv.destroyAllWindows()

### Aplicando umbral 180
Vemos algo interesante, el cuerpo y parte de la aguja que tiene el personaje han desaparecido, sin embargo las cabeza y la punta de la aguja, las cuales son las secciones con mayor intensidad, se mantuvieron de color blanco.

In [8]:
image = cv.imread('Hornet.jpg',cv.IMREAD_GRAYSCALE)

ret_val,image_umbral = cv. threshold(image, 180,
                             255,
                             cv.THRESH_BINARY)
cv.imshow('Original_image',image)
cv.imshow('Image_umbral',image_umbral)
cv.waitKey(0)
cv.destroyAllWindows()

### Aplicando umbral   0 < q < 255 a una imagen binarizada 
En los siguientes ejemplos trabajaremos con imagenes binarizadas, para observar que es lo que ocurre en las fronteras de intensidad, 0 y 255 cuando se aplican ciertos umbrales.

En este caso vemos el resultado de aplicar diferentes menores a 255, se puede observar que no ocurre nada, lo que tiene sentido con la teoría.

In [9]:
image = cv.imread('Hornet.jpg',cv.IMREAD_GRAYSCALE)

# Binarizamos la imagen original a un cierto umbral
_,img_bin = cv. threshold(image, 30,
                             255,
                             cv.THRESH_BINARY)

# Vemos que ocurre en los limites de los umbrales cuando tenemos pixeles o completamente negros p = 0, o 
# pixeles completamente blancos p = 255, como es el caso de una imagen binarizada:
_,img_um_100 = cv. threshold(img_bin, 100,
                             255,
                             cv.THRESH_BINARY)

cv.imshow('Original',image)
cv.imshow('Image_bin',img_bin)
cv.imshow('Image_umbral_255',img_um_100)
cv.waitKey(0)
cv.destroyAllWindows()

### Aplicando umbral q = 255 a una imagen binarizada
Notamos que la imagen se vuelve completamente negra, lo que corresponde a un caso correcto del análisis teorico.

In [10]:
_,img_um_255 = cv. threshold(img_bin, 255,
                             255,
                             cv.THRESH_BINARY)
cv.imshow('Image_bin',img_bin)
cv.imshow('Image_umbral_255',img_um_255)
cv.waitKey(0)
cv.destroyAllWindows()

# Problema 4 - Autocontraste modificado
Antes de hablar del autocontraste modificado, es importante comprender que es el contraste, posteriormente definiremos que es el autocontraste y la diferencia entre el autocontraste modificado y el no modificado.
##### Definiendo un poco el concepto de contraste
Cuando se hace mencion de **contraste**, lo primero que se nos debe de venir a la mente es que tan distinto es el negro más negro, del blanco más blanco en una imagen.

Un poco más formal, el contraste lo podemos definir como la diferencia que existe entre el valor de intensidad más grande $p_\text{h}$, menos el valor de intensidad más pequeño $p_\text{l}$ (suponiendo una escala de grises).
$$ 
\text{Contraste} = p_\text{h} - p_\text{l}
$$

Sabiendo esto, si una imagen $A$ tiene pixeles con valores dentro del intervalo $[100,150]$, mientras que una imagen $B$ tiene pixeles con valores dentro del intervalo $[50,210]$, podemos afirmar que $A$ tiene menos contraste que $B$, dado que hay una mayor distancia entre las intensidades más bajas con las más altas.

Entre más contraste tenga una imagen, los detalles en esta misma seran más claros, mientras que con un contraste más bajo, la imagen se verá menos detallada o apagada. Esto debido a que como el contraste es la distancia entre los extremos de las intensidades, entonces los colores tienden a parecerse más unos con los otros, a diferencia de si usaramos toda la escala de grises que disponemos.
A este ultimo tipo de imagenes, se les conoce como **full-contrast** 

##### Entonces, ¿Qué es el auto contraste?
El autocontraste nos permite realizar un mapeo de los pixeles, para que estos ocupen todo el rango de itensidad (en escala de grises, toda la escala de grises). Por lo tanto, el autocontraste **solo** lo usaremos cuando una imagen no abarca todo el rango de intensidad disponible, como veremos en la formula más adelante.

Para realizar este mapeo, primero debemos tener claro nuestro rango de intensidad al que queremos mapear $[p_\text{min},p_\text{max}]$ , suponiendo que queremos la intensidad minima y maxima posible tendríamos $[0,255]$, y el rango de intensidad correspondiente a la imagen $[p_\text{l},p_\text{h}]$, posteriormente aplicamos a cada pixel la siguiente función:
$$
f_\text{ac}(a) = p_\text{min} + (a - p_\text{l}) \cdot \frac{p_\text{max}-p_\text{min}}{p_\text{h}-p_\text{l} }
$$
Analizando un poco la expresión, el termino $\frac{p_\text{max}-p_\text{min}}{p_\text{h}-p_\text{l}}$ es siempre un valor positivo, mas aún, suponiendo que el mapeo siempre es con la finalidad de aumentar el contraste (aunque no necesariamente siempre se hace para aumentar el contraste), $ p_\text{max}-p_\text{min} > p_\text{h}-p_\text{l}$, por lo tanto el termino completo siempre es mayor a 1. Este factor es la constante "lineal" con la cuál vamos a escalar los pixeles.

Además tenemos el termino $(a-p_\text{l})$, este termino, nos determina diferencia de intensidad que hay entre el pixel y el pixel con menor intensidad. Esta diferencia posteriormente se va a escalar, ya que para integrar este nuevo pixel a una escala de mayor contraste, usamos esta relación lineal. 

Aplicando la simplificación que se recomendo en clase, de tomar el rango de intensidad para imagenes full-contrast $[0,255]$ llegamos a:
$$
f_\text{ac} = (a - p_\text{l}) \cdot \frac{255}{p_\text{h}-p_\text{l} }
$$

##### Si el autocontraste ya soluciona el problema, ¿para que nos sirve el autocontraste modificado?
El autocontraste parece ser una gran herramienta, sin embargo, supongamos que el $90\%$ de los pixeles de nuestra imagen se encuentran en el rango de intensidad $[100,150]$, mientras que un $6\%$ se encuentra en el rango $[20,25]$ y el otro $4\%$ se encuentra en el rango $[201,215]$, si nosotros realizamos un autocontraste a nuestra imagen, veremos como nuestra imagen pierde definición, debido a que estamos "estirando" nuestro rango de intensidad basado en valores atipicos en los extremos, cuando en realidad el $90\%$ de nuestra imagen se encuentra en un rango muy diferente a nuestros valores tipicos.

La forma de solucionar este problema, es similar a como se plantean los gráficos de caja, los cuales a su vez, tratan de despejar las estimaciones importantes de los datos atipicos, separando el conjunto de datos en cuartiles.

Para este método, deberemos ubicar 2 nuevos valores, los correspondientes a la intensidad del cuartil inferior $q_l$ y el cuartil superior $q_h$. De forma similar al autocontraste, pero separando los quartiles, mapeamos los pixeles con la siguiente función:

$$ 
f_\text{mac}(a) =
\begin{cases}
p_\text{min} & \text{si } a \le q_l  \\
p_\text{min} + (a - p_\text{l}) \cdot \frac{p_\text{max}-p_\text{min}}{p_\text{h}-p_\text{l} } & \text{si } q_l < a <q_h \\
p_\text{max} & \text{si } a \ge q_h
\end{cases}
$$

##### Cuando usarla y cuando no
Es interesante usar un autocontraste cuando el contraste de la imagen es muy bajo y no se alcanzan a notar los detalles de la imagen. Sin embargo, si una imagen ya tiene un contraste alto, podríamos reducir su contraste para intentar apagar un poco la intensidad de los colores.

Esta función no tiene sentido el aumentar de contraste en imagenes con muy poca variación de valores de pixeles, por ejemplo una imagen binarizada con 0 y 255, debido a que los valores inferior y superior ya son el máximo y el mínimo.

### Ejemplo 1 - Faro/niebla
Podemos apreciar como el aumento de contraste resalta el faro, además que se puede apreciar de una mejor manera ciertos detalles de las capas de la niebla que no se notaban por el contraste bajo que tenía.

In [11]:
image = Image.open("Faro_gray.jpg")

# Aplicamos el auto contraste
img_ac = ImageOps.autocontrast(image)

# Desplegamos las imagenes, esperar a que se abra su visualizador de fotos
image.show(title='Faro')
img_ac.show(title='Faro_contrast')

### Ejemplo 2 - Bosque
Vemos como las ramas alcanzan una mejor definición, en especial los reflejos del lago y los arboles de la esquina inferior derecha

In [ ]:
image = Image.open("Forest_gray.jpg")

img_mac= ImageOps.autocontrast(image)

image.show(title='Forest')
img_mac.show(title='Forest_contrast')

# Problema 5 - Operador Logaritmo
Cuando nosotros pensamos en la función logaritmo, es extremadamente común pensar en un crecimiento lento. Dada esta propiedad, es muy utilizada en diferentes areas, por ejemplo, si se desea análizar un conjunto de datos, los cuales estan muy dispersos entre si, o si existen valores extremos atipicos, esto debido a que la función logaritmo transforma los valores grandes a valores mucho más pequeños, y los valores pequeños no los modifica tanto.

El uso de logaritmos para el procesamiento de imagenes, puede resultar útil para controlar una gran variación en las intensidades de una imagen, consiguiendo que no se encuentren diferencias tan bruscas en las intensidades de la imagen. 

Esta operación se cálcula de la forma:
$$
Q(i,j) = c\log(1+I_p)
$$
Donde 
- $I_p$ corresponde al valor de la intensidad del pixel a modificar
- c es el valor de escala, ya que ocupamos ajustar la función logaritmo para que nuestros valores sean mapeados adecuadamente en el rango $[0,255]$.
El calculo de c se realiza lo podemos realizar con un despeje, suponiendo que al aplicar la operación logaritmo conseguimos la intensidad máxima justo cuando aplicamos este operador a nuestro pixel con mayor intensidad $I_p==$ el valor del pixel con mayor intensidad en la imagen.
Realizando el despeje tenemos:
$$  
\begin{align*}
Q(i,j) &= c\log(1+I_p)\\
255 &= c\log(1+I_\text{pmax})\\
c &= \frac{255}{\log(1+I_\text{pmax})}
\end{align*}
$$
##### Cuando es interesante realizazr esta operación?
Como mencionamos anteriormente, aplicar la operación logaritmo reduce valores muy grandes, y los pequeños los modifica muy poco, teniendo esto en cuenta, imagenes con un alto contraste o con muy bajo contraste, pueden ser buenas candidatas para aplicar esta operación.

Si la imagen ya cuenta con un contraste bueno, puede no mejorar la imagen, o puede empeorar la calidad de esta ya que estaríamos transformando una imagen que ya contaba con una buena definición.

### Ejemplo 1 - Aplicando operador logaritmo a una imagen de alto contraste
En esta imagen, podemos notar como al aplicar el operador logaritmo, muchas zonas de la imagen se volvieron grises, pero curiosamente hay unos trazos o pixeles un poco raros en la copa izquierda, estos corresponden a zonas que probablemente se deseaba amplificar los tonos oscuros, por lo que tal vez remarcaron el color oscuro un poco más.

In [ ]:
image = cv.imread("Cups.jpg",cv.IMREAD_GRAYSCALE)

# Convertimos a punto flotante las intensidades
img = image.astype(np.float32)

# Calculamos la constante de escalado
c = 255 / np.log(1 + np.max(img))

# Aplicamos el operador logaritmo a las intensidades
image_log = c * np.log(1 + img) 

# Regresamos las intensidades en forma de enteros
image_log = np.uint8(np.clip(image_log,0,255))

cv.imshow("Original",image)
cv.imshow("image_log",image_log)
cv.waitKey(0)
cv.destroyAllWindows()

### Ejemplo 2 - Aplicando operador logaritmo a una imagen con un contraste adecuado
Podemos observar, como al aplicar el operador logaritmo a una imagen que actualmente tiene un contraste adecuado, se ve perjudicada, y no consigue ni si quiera la calidad de la imagen de contraste inicial (ya que su contraste es mucho menor).

In [ ]:
image = Image.open("Faro_gray.jpg")

# Aplicamos el auto contraste para darle un contraste bueno a esta foto
img_ac = ImageOps.autocontrast(image)

# Transformamos los objetos pil a un arreglo tipo numpy
image = np.array(image)
img_ac = np.array(img_ac)

# Ahora aplicamos el operador logaritmo de la misma forma que en el problema anterior

img_ac_f = img_ac.astype(np.float32)

c = 255 / np.log(1 + np.max(img_ac_f))
image_log = c * np.log(1 + img_ac_f) 

image_log = np.uint8(np.clip(image_log,0,255))

# Desplegamos las imagenes para compararlas
cv.imshow("Original",image)
cv.imshow("image_contrast",img_ac)
cv.imshow("image_log",image_log)
cv.waitKey(0)
cv.destroyAllWindows()

# Problema 6 - Operador Exponencial

Su efecto principal es realzar los píxeles con intensidades altas.

### Formula

$$
Q(i,j) = c(b^{P(i,j)} - 1)
$$

Donde $b$ es la base (generalmente un valor ligeramente mayor que 1) y $c$ es un factor de escala para asegurar que el valor maximo de $Q(i,j)$ sea 255.

### Usos

Funciona bien en:
- Imagenes con zonas claras que se quieran resaltar.
- Aumento drastico de luminisidad.
- Cuando se desea revertir los efectos del operador logaritmo

No conviene usarlo en:
- Imagenes oscuras.
- Imagenes con ruido en zonas brillantes.

### Ejemplo 1

In [ ]:
image = cv.imread('Castillo.jpg',cv.IMREAD_GRAYSCALE)

b = 1.02 # Base un poco mayor que 1
c = 255 / (b**255 - 1)

exp_img = c * (np.power(b, image) - 1)
exp_img = np.clip(exp_img, 0, 255).astype(np.uint8)

cv.imshow("Original", image)
cv.imshow("Exponencial", exp_img)
cv.waitKey(0)
cv.destroyAllWindows()


### Ejemplo 2 - Inverso del operador logaritmo
Se retoma el ejemplo 1 del problema 5

In [ ]:
image = cv.imread("Cups.jpg",cv.IMREAD_GRAYSCALE)

# Convertimos a punto flotante las intensidades
img = image.astype(np.float32)

c = 255 / np.log(1 + np.max(img))
image_log = c * np.log(1 + img) 

# Regresamos las intensidades en forma de enteros
image_log = np.uint8(np.clip(image_log,0,255))
b = 1.02 # Base un poco mayor que 1
c = 255 / (b**255 - 1)

img_aux = image_log

exp_img = c * (np.power(b, img_aux) - 1)
exp_img = np.clip(exp_img, 0, 255).astype(np.uint8)

cv.imshow("Original",image)
cv.imshow("image_log",image_log)
cv.imshow("Exponencial", exp_img)
cv.waitKey(0)
cv.destroyAllWindows()

# Problema 7 - Correción Gamma

### Que es?

Esta operacion ajusta la luminosidad de una imagen, se utilza para compensar el como los dispositivos capturan o muestran la intensidad de la luz.

### Formula

La formula mas usada es:
$$
I_{out} = \left(  \frac{I_{in}}{I_{max}}\right)^{\gamma} \cdot I_{max}
$$

### Usos

Funciona bien en:
- Funciona bien en fotos muy oscuras.
- Hacer que la imagen se vea mas natural.
- Preparar imagenes para impresion.

No conviene usarla en:
- Imagenes ya correctamente iluminadas, puede distorsionar los colores.
- Imagenes con mucho ruido, al aclararlas se genera aun mas ruido.

In [ ]:
def gamma_correction(image, gamma=1.0):
    img_norm = image.astype(np.float32) / 255.0
    corrected = np.power(img_norm, gamma)
    return np.uint8(corrected * 255.0)


img = cv.imread('Castillo.jpg',cv.IMREAD_GRAYSCALE)

# Ejemplos con distintos gammas
gamma_dark = gamma_correction(img, 1.5) # oscurece
cv.imshow("Gamma=1.5", gamma_dark)

gamma = gamma_correction(img) # no cambia
cv.imshow("Gamma=1.0", gamma)

gamma_bright = gamma_correction(img, 0.5) # aclara
cv.imshow("Gamma=0.5", gamma_bright)

cv.waitKey(0)
cv.destroyAllWindows()